### 1. How you can call C function from python
There are three key methods developers use to call C functions from their python code - **ctypes, SWIG and Python/C API**. Each method comes with it’s own merits and demerits.

###### Firstly, why would you want to interface C with Python?

A few common reasons are :

1. You want speed and you know C is about 50x faster than Python.
2. Certain legacy C libraries work just as well as you want them to, so you don’t want to rewrite them in python.
3. Certain low level resource access - from memory to file interfaces.

###### CTypes
[CTypes](https://docs.python.org/2/library/ctypes.html) <br>
The Python ctypes module is probably the easiest way to call C functions from Python. The ctypes module provides C compatible data types and functions to load DLLs so that calls can be made to C shared libraries without having to modify them. The fact that the C side needn’t be touched adds to the simplicity of this method.

In [ ]:
## Simple C code to add two numbers, save it as add.c
#include <stdio.h>

int add_int(int, int);
float add_float(float, float);

int add_int(int num1, int num2){
    return num1 + num2;
}

float add_float(float num1, float num2){
    return num1 + num2;
}

###### Next compile the C file to a .so file (DLL in windows) This will generate an adder.so file.

In [ ]:
#For Linux
$  gcc -shared -Wl,-soname,adder -o adder.so -fPIC add.c

#For Mac
$ gcc -shared -Wl,-install_name,adder.so -o adder.so -fPIC add.c

In [ ]:
## Now in your python code -

from ctypes import *

#load the shared object file
adder = CDLL('./adder.so')

#Find sum of integers
res_int = adder.add_int(4,5)
print "Sum of 4 and 5 = " + str(res_int)

#Find sum of floats
a = c_float(5.5)
b = c_float(4.1)

add_float = adder.add_float
add_float.restype = c_float
print "Sum of 5.5 and 4.1 = ", str(add_float(a, b))

### 2. How can you debug .py file

In [ ]:
## Running from commandline
$ python -m pdb my_script.py

"""
It would cause the debugger to stop the execution on the first statement it finds. 
This is helpful if your script is short. 
You can then inspect the variables and continue execution line-by-line.
"""

In [ ]:
##Running from inside a script
"""
You can set break points in the script itself so that you can inspect 
the variables and stuff at particular points. 
This is possible using the pdb.set_trace() method. Here is an example:
"""
import pdb

def make_bread():
    pdb.set_trace()
    return "I don't have time"

print(make_bread())

##### Debug Commands:

    c: continue execution
    w: shows the context of the current line it is executing.
    a: print the argument list of the current function
    s: Execute the current line and stop at the first possible occasion.
    n: Continue execution until the next line in the current function is reached or it returns.


### 3. Generators
There are three parts namely:

1. Iterable
2. Iterator
3. Iteration

#### 3.1. Iterable
An iterable is any object in Python which has an $__iter__ $ or a $__getitem__ $ method defined which returns an iterator or can take indexes (You can read more about them [here](https://stackoverflow.com/questions/20551042/whats-the-difference-between-iter-and-getitem/20551346#20551346)). In short an iterable is any object which can provide us with an iterator. So what is an iterator?

#### 3.2. Iterator
An iterator is any object in Python which has a next (Python2) or $__next__ $ method defined. That’s it. That’s an iterator. Now let’s understand iteration.

#### 3.3. Iteration
In simple words it is the process of taking an item from something e.g a list. When we use a loop to loop over something it is called iteration. It is the name given to the process itself. Now as we have a basic understanding of these terms let’s understand generators.

#### 3.4. Generators
**Generators are iterators, but you can only iterate over them once.** It’s because they do not store all the values in memory, they generate the values on the fly. You use them by iterating over them, either with a ‘for’ loop or by passing them to any function or construct that iterates. Most of the time generators are implemented as functions. However, they do not return a value, they yield it. Here is a simple example of a generator function:

In [3]:
def generator_function():
    for i in range(10):
        yield i

for item in generator_function():
    print(item)

0
1
2
3
4
5
6
7
8
9


It is not really useful in this case. Generators are best for calculating large sets of results (particularly calculations involving loops themselves) where you don’t want to allocate the memory for all results at the same time. **Many Standard Library functions that return lists in Python 2 have been modified to return generators in Python 3 because generators require fewer resources.**

Here is an example generator which calculates fibonacci numbers:

In [ ]:
# generator version
def fibon(n):
    a = b = 1
    for i in range(n):
        yield a
        a, b = b, a + b

# Please dont run this cell, as it will stop the execution of notebook
for x in fibon(100000):
    print(x)

This way we would not have to worry about it using a lot of resources. However, if we would have implemented it like below:


It would have used up all our resources while calculating a large input(100000).

In [ ]:
def fibon(n):
    a = b = 1
    result = []
    for i in range(n):
        result.append(a)
        a, b = b, a + b
    return result

**You can iterate over generator only once**

In [ ]:
def generator_function():
    for i in range(3):
        yield i

gen = generator_function()
print(next(gen))
# Output: 0
print(next(gen))
# Output: 1
print(next(gen))
# Output: 2
print(next(gen))
# Output: Traceback (most recent call last):
#            File "<stdin>", line 1, in <module>
#         StopIteration

As we can see that after yielding all the values next() caused a StopIteration error. Basically this error informs us that all the values have been **yielded.**

**Few built-in data types in Python also support iteration**

In [ ]:
my_string = "Jyotirmoy"
next(my_string)
# Output: Traceback (most recent call last):
#      File "<stdin>", line 1, in <module>
#    TypeError: str object is not an iterator

The error says that str is not an iterator. Well it’s right! **It’s an iterable but not an iterator.** This means that it supports iteration but we can’t iterate over it directly. Below is the example of how you can iterate over.

In [7]:
my_string = "Jyotirmoy"
my_iter = iter(my_string)
print(next(my_iter))

J


### 4. $__slots__$ Magic
In Python every class can have instance attributes. By default Python uses a **dict** to store an object’s instance attributes. This is really helpful as it allows setting arbitrary new attributes at runtime.

However, for small classes with known attributes it might be a bottleneck. The dict wastes a lot of RAM. Python can’t just allocate a static amount of memory at object creation to store all the attributes. Therefore it sucks a lot of RAM if you create a lot of objects (I am talking in thousands and millions). Still there is a way to circumvent this issue. It involves the usage of __slots__ to tell Python not to use a dict, and only allocate space for a fixed set of attributes. Here is an example with and without __slots__:

In [ ]:
## Without __slots__:

class MyClass(object):
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        self.set_up()
    # ...

In [ ]:
## With __slots__:

class MyClass(object):
    __slots__ = ['name', 'identifier']
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        self.set_up()
    # ...

The second piece of code will reduce the burden on your RAM. Some people have seen almost 40 to 50% reduction in RAM usage by using this technique.

### 5. Decorators
Decorators are a significant part of Python. In simple words: they are functions which modify the functionality of other functions. They help to make our code shorter and more Pythonic.

#### 5.1 First Decorator:


In [1]:
def a_new_decorator(a_func):

    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")

        a_func()

        print("I am doing some boring work after executing a_func()")

    return wrapTheFunction

def a_function_requiring_decoration():
    print("I am the function which needs some decoration to remove my foul smell")

a_function_requiring_decoration()

a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)

a_function_requiring_decoration()


I am the function which needs some decoration to remove my foul smell
I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


#### 5.2 Little modified version with @anywhere


In [2]:
@a_new_decorator
def a_function_requiring_decoration():
    print("I am the function which needs some decoration to remove my foul smell")
    
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


#### 5.3 Blueprint
@wraps takes a function to be decorated and adds the functionality of copying over the function name, docstring, arguments list, etc. This allows to access the pre-decorated function’s properties in the decorator.

In [3]:
from functools import wraps
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated

@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())
# Output: Function is running

can_run = False
print(func())
# Output: Function will not run

Function is running
Function will not run


#### 5.4 Use-cases:
Now let’s take a look at the areas where decorators really shine and their usage makes something really easy to manage.

##### 5.4.1 Authorization
Decorators can help to check whether someone is authorized to use an endpoint in a web application. They are extensively used in Flask web framework and Django. Here is an example to employ decorator based authentication:

In [4]:
from functools import wraps

def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated